In [1]:
# from podcast_ad_skipper.Spectrogram_Conversion import create_spectrogram
import pandas as pd
from podcast_ad_skipper.google_cloud import *
import librosa
from podcast_ad_skipper.params import *
from podcast_ad_skipper.data_preparation import *

from google.auth.exceptions import GoogleAuthError
from google.cloud import storage
from google.oauth2 import service_account
from google.cloud import bigquery


In [2]:
storage_client = auth_gc()

Authenticated successfully! ✅


In [3]:
prefixes =GCP_PREFIXES[0]
bucket_name = BUCKET_NAME

In [4]:
file_list = retrieve_files_in_folder(storage_client, bucket_name, prefixes)

In [5]:
# open_file = open_gcs_file(file_list[0])

In [6]:
features = get_features_model(file_list)

Processing files: total 5837


In [7]:
import pandas as pd
from podcast_ad_skipper.main import transform_features_into_dataframe

In [8]:
data = transform_features_into_dataframe(features)
data

,spectrogram_np_arrays,labels,seconds,durations,podcast_names
0,"[[-48.69738, -54.582626, -64.74443, -66.06518,...",0,1000,5864,billionairepersonalitydisorder
1,"[[-68.663124, -66.371155, -61.690292, -64.7245...",0,1001,5864,billionairepersonalitydisorder
2,"[[-67.3631, -70.561775, -67.293076, -50.00025,...",0,1002,5864,billionairepersonalitydisorder
3,"[[-38.44786, -44.348087, -66.34653, -59.608376...",0,1003,5864,billionairepersonalitydisorder


In [10]:
bq_client = auth_gc_bigquery()


Authenticated successfully with BigQuery! ✅


In [11]:
table_id = f"{GCP_PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}"

In [12]:
append_arrays_to_bq(data, bq_client, table_id)

Authenticated successfully with BigQuery! ✅


ArrowInvalid: Can only convert 1-dimensional array values

In [ ]:
append_arrays_to_bq(bq_client, np_array, table_id)